App Profiles in Play and App Stores
=============================
The goal of this project is to go through the data in the Google Play Store and Apple App Store to uncover app profiles.

Opens data and separates it into header and body: 

In [1]:
from csv import reader

opened_file=open('googleplaystore.csv')
read_file=reader(opened_file)
play_store_data_full=list(read_file)
play_store_header=play_store_data_full[0]
play_store_data=play_store_data_full[1:]


opened_file=open('AppleStore.csv')
read_file=reader(opened_file)
app_store_data_full=list(read_file)
app_store_header=app_store_data_full[0]
app_store_data=app_store_data_full[1:]

Prints the contents of an opened file:

In [2]:
def print_data(set, start, end, print_rows_columns=False):
    set_slice=set[start:end]
    for row in set_slice:
        print(row)
        print('\n')
    if print_rows_columns:
        print('Number of rows: ', len(set))
        print('Number of columns: ',  len(set[0]))


Testing function:

In [3]:
print_data(play_store_data, 0, 1, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows:  10841
Number of columns:  13


The following entry has a rating of 19, whereas the max for the Play Store is 5:

In [4]:
print_data(play_store_data, 10472, 10473, True)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Number of rows:  10841
Number of columns:  13


In [5]:
del play_store_data[10472]

Looking through the Play Store data manually, we will find duplicate entries, the following code checks for duplicate entries of Instagram:

In [6]:
for row in play_store_data:
    name=row[0]
    if name=='Instagram':
        print(row)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Next is to scan for duplicate entries, which will be cleaned based on number of reviews:

In [7]:
duplicates=[]
unique=[]
for row in play_store_data:
    name=row[0]
    if name not in unique:
        unique.append(name)
    elif name in unique:
        duplicates.append(name)
print(len(duplicates))

1181


The following code goes through the Play Store data to add the entries with the highest number of reviews to a dictionary:

In [8]:
max_reviews={}
for row in play_store_data:
    name=row[0]
    n_reviews=float(row[3])
    if (name in max_reviews) and (max_reviews[name]<n_reviews):
        max_reviews[name]=n_reviews
    elif name not in max_reviews:
        max_reviews[name]=n_reviews
print(len(max_reviews))

9659


In [9]:
ps_cleaned=[]
already_added=[]

for row in play_store_data:
    name=row[0]
    n_reviews=float(row[3])
    
    if (max_reviews[name]==n_reviews) and (name not in already_added):
        ps_cleaned.append(row)
        already_added.append(name)
        
print(len(ps_cleaned))

print_data(ps_cleaned, 0, 5, True)

9659
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows:  9659
Number of columns:  13


Next, to check for apps with more than 3 non-English characters. The non-English strings will be grouped and discarded via their ASCII index. The following function checks whether or not a string is using mostly English characters:

In [10]:
def is_english(string: str):
    counter=0
    for character in string:
        if ord(character)>127:
            counter+=1
    if counter>3:
        return False
    else:
        return True
print(is_english('Instagram'))
print(is_english('爱奇艺PPS'))
print(is_english('Instachat 😜'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
True
False


The following code adds the names of the English apps to a separate list, there might be some non-English apps that are included however for the purposes of this project this will be enough.

In [11]:
ps_english_apps=[]
for row in ps_cleaned:
    name=row[0]
    if is_english(name):
        ps_english_apps.append(row)
        
as_english_apps=[]
for row in app_store_data:
    name=row[1]
    if is_english(name):
        as_english_apps.append(row)
        
print(len(ps_english_apps))
print(len(as_english_apps))

9614
6183


The following code groups free apps in a separate list, as they are the most commonly downloaded:

In [12]:
android_apps=[]
ios_apps=[]
for row in ps_english_apps:
    price=row[7]
    if price=='0':
       android_apps.append(row)
        
for row in as_english_apps:
    price=row[4]
    if price=='0.0':
        ios_apps.append(row)
print(len(android_apps))
print(len(ios_apps))

8864
3222


Next we will group by genre. The next two functions are for generating percentages of each genre in a given set:

In [13]:
def freq_table(dataset, index):
    table={}
    total_apps=0
    for row in dataset:
        total_apps+=1
        value=row[index]
        if value in table:
            table[value]+=1
        else:
            table[value]=1
            
    table_percentages={}
    for key in table:
        percentage=(table[key]/total_apps)*100
        table_percentages[key]=percentage
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    display_table = []
    for key in table:
        tuple_table = (table[key], key)
        display_table.append(tuple_table)

    sorted_table = sorted(display_table, reverse = True)
    for entry in sorted_table:
        print(entry[1], ':', entry[0])

We will use the display_table function on both the Play Store and App Store apps:

In [14]:
display_table(android_apps, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

Filler

In [15]:
display_table(ios_apps, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Next we will find out which iOS apps are most popular in a particular genre by looking at the number of ratings:

In [16]:
ios_genres=freq_table(ios_apps, -5)
for genre in ios_genres:
    total=0
    genre_total=0
    for app in ios_apps:
        genre_app=app[-5]
        if genre_app==genre:
            user_ratings=float(app[5])
            total+=user_ratings
            genre_total+=1
    avg=total/genre_total
    print(genre,':', avg)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Same thing for Play Store:

In [17]:
android_genres=freq_table(android_apps, 1)
for genre in android_genres:
    total=0
    genre_total=0
    for app in android_apps:
        genre_app=app[1]
        if genre_app==genre:
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total+=float(n_installs)
            genre_total+=1
    avg=total/genre_total
    print(genre,':', avg)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_